In [ ]:
# importing dependencies
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
# download nltk from nltk memory
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# printing stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
# Data collection and preprocessing
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/My Drive/Colab Notebooks/'

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/news_train.csv')

In [ ]:
data.shape

(20800, 5)

In [ ]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
# check if the values are missing in the dataset
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
data = data.fillna('')

In [ ]:
data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
# For prediction we will use 'title' and 'author'. We are not going to use text column coz it is very large
# merging 'author' and 'title' in new column 'content'
data['content'] = data['author'] + ' ' + data['title']

In [ ]:
data['content']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [ ]:
# diving the data into x(independent variable) and y(dependent variable)
x = data.drop(columns = 'label', axis = 1)
y = data['label']

In [ ]:
x.shape

(20800, 5)

In [ ]:
y.shape

(20800,)

In [ ]:
x.head()

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...


In [ ]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [ ]:
# Now, doing stemming
# Stemming: Stemming is the process of reducing a word to its root word. Example: acting, acted, acts - act
# It will remove the prefix and suffix and converts to root word coz with prefix and suffix, it doesm't give meaning to our text

In [ ]:
# bringing stemming function
port_stem = PorterStemmer()

In [ ]:
# creating a stemming function to apply in our data
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content) # here the values excluding small a-z and large A-Z (excluding kine vani square bracket lagaxa) i.e content variable ma vako
                                                        #  numbers, characters,commaetc harulai space le sub - substitute (replace) garni coz they are useless
  stemmed_content = stemmed_content.lower() # lower case ma lagne
  stemmed_content = stemmed_content.split() # split garni
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content) # join garxa
  return stemmed_content 

In [ ]:
# Apply the stemming function that we just created above
data['content'] = data['content'].apply(stemming)

In [ ]:
data['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [ ]:
# separating the data and label in x and y
x = data['content'].values
y = data['label'].values

In [ ]:
# Converting the textual data into numerical data using TfidfVectorizer as computers dont understand the textual words
# TfidfVectorizer: Tf stands for term frequency, idf stands for inverse document frequency. It counts the number of words repeating in a document
# All the words from x i.e 'content' will be converted to their respective feature vectors(simply numbers)

vectorizer = TfidfVectorizer()
vectorizer.fit(x)

x = vectorizer.transform(x)

In [ ]:
# lets check the numerical form of x i.e content
print(x)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [ ]:
# Now, we feed these transformed text i.e feature vectors to our ml model to do predictions
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 2)
# 0.2 i.e 20% test data, 80% train data, stratify - helps to separate fake and real news in eqla proportion, random_state = 2

In [ ]:
model = LogisticRegression()

In [ ]:
# train our model using fit function 
model.fit(x_train, y_train)

LogisticRegression()

In [ ]:
# As our model is trained, lets find the accuracy score of our trained model with original train label
# accuracy score of training data
x_train_prediction = model.predict(x_train) # our models prediction of data
training_data_accuracy = accuracy_score(x_train_prediction, y_train) # comparing how accuracte (in %)our models prediction is with original train data

In [ ]:
# printing accuracy score
print(training_data_accuracy)

0.9873798076923077


In [ ]:
# We can see that our trained model predicts with nearly 99% accuracy while comparing with our original labelled data. So, it is best fit model.

In [ ]:
# finding accuracy on our test data as our real work is done on test data
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction, y_test)

In [ ]:
print(test_data_accuracy)

0.9774038461538461


In [ ]:
# while testing our models test data prediction with our original labelled y test, we found that our model is nearly 98% accurate with the original labelled y test data.
# Our model predicts data with 98 percent accuracy

In [ ]:
# Now lets predict our news is fake or real by using simple code snippet
x_new = x_test[6] # x_test[0] i.e first row data
prediction = model.predict(x_new)

In [ ]:
print(prediction)

[0]


In [ ]:
# As we know 1 : Fake news
#            0 : Real news
if (prediction[0] == 0): 
  print('The news is real')
else:
  print('The news is fake')


The news is real


In [ ]:
# Now, check from original labelled y data that news is fake or real
print(y_test[6])

0


In [ ]:
# Here, the predicted and real values are same. We found that the news is real as output is '0'.